In [12]:
import cv2
import mediapipe as mp
import random
import numpy as np
import time

mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.7)
mp_draw = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0)

def count_fingers_precise(hand_landmarks, hand_label='Right'):
    tip_ids = [4, 8, 12, 16, 20]
    fingers = []
    landmarks = hand_landmarks.landmark
    
    if hand_label == 'Right':
        if landmarks[tip_ids[0]].x < landmarks[tip_ids[0]-1].x:
            fingers.append(1)
        else:
            fingers.append(0)
    else: 
        if landmarks[tip_ids[0]].x > landmarks[tip_ids[0]-1].x:
            fingers.append(1)
        else:
            fingers.append(0)
    
    for id in range(1,5):
        if landmarks[tip_ids[id]].y < landmarks[tip_ids[id]-2].y:
            fingers.append(1)
        else:
            fingers.append(0)
    return sum(fingers)

def classify_rps(fingers):
    if fingers == 0 or fingers == 1:
        return 'Rock'
    elif fingers == 2:
        return 'Scissors'
    elif fingers >= 4:
        return 'Paper'
    else:
        return 'Unknown'

def countdown(cap, seconds=3):
    start_time = time.time()
    while time.time() - start_time < seconds:
        ret, frame = cap.read()
        if not ret:
            continue
        elapsed = int(seconds - (time.time() - start_time) + 1)
        cv2.putText(frame, str(elapsed), (280,240), cv2.FONT_HERSHEY_SIMPLEX, 5, (0,0,255), 7)
        cv2.imshow('Rock Paper Scissors', frame)
        if cv2.waitKey(1) & 0xFF == 27:
            return False
    return True

def determine_winner(player, computer):
    if player == computer:
        return 'Tie'
    elif (player == 'Rock' and computer == 'Scissors') or \
         (player == 'Scissors' and computer == 'Paper') or \
         (player == 'Paper' and computer == 'Rock'):
        return 'Player'
    else:
        return 'Computer'

def play_rps():
    player_score = 0
    computer_score = 0

    while True:
        if not countdown(cap, 3):
            break

        computer_choice = random.choice(['Rock','Paper','Scissors'])

        player_choice = 'Unknown'
        winner = 'Waiting...'
        start = time.time()

        while True:
            ret, frame = cap.read()
            if not ret:
                continue
            img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            result = hands.process(img_rgb)

            if result.multi_hand_landmarks:
                for hand_landmarks, hand_handedness in zip(result.multi_hand_landmarks, result.multi_handedness):
                    hand_label = hand_handedness.classification[0].label
                    fingers = count_fingers_precise(hand_landmarks, hand_label)
                    player_choice = classify_rps(fingers)
                    mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                if player_choice in ['Rock','Paper','Scissors']:
                    winner = determine_winner(player_choice, computer_choice)
                    if winner == 'Player':
                        player_score += 1
                    elif winner == 'Computer':
                        computer_score += 1
                    break

            cv2.putText(frame, f'Your choice: {player_choice}', (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,255), 2)
            cv2.putText(frame, f'Computer chose: {computer_choice if player_choice in ["Rock","Paper","Scissors"] else "???"}', (50,400), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
            cv2.putText(frame, f'Score - Player: {player_score} Computer: {computer_score}', (50,450), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2)
            cv2.imshow('Rock Paper Scissors', frame)
            if cv2.waitKey(1) & 0xFF == 27:
                cap.release()
                cv2.destroyAllWindows()
                return

        start_display = time.time()
        while time.time() - start_display < 2:
            ret, frame = cap.read()
            if not ret:
                continue
            cv2.putText(frame, f'Player: {player_choice}', (50,150), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,0,0), 3)
            cv2.putText(frame, f'Computer: {computer_choice}', (50,250), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,0,255), 3)
            cv2.putText(frame, f'Winner: {winner}', (50,350), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,255,0), 3)
            cv2.putText(frame, f'Score - Player: {player_score} Computer: {computer_score}', (50,450), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2)
            cv2.imshow('Rock Paper Scissors', frame)
            if cv2.waitKey(1) & 0xFF == 27:
                cap.release()
                cv2.destroyAllWindows()
                return

        if player_score >= 5 or computer_score >= 5:
            break

    cap.release()
    final_frame = 255 * np.ones((480,640,3), np.uint8)
    cv2.putText(final_frame, 'FINAL SCORE', (180,150), cv2.FONT_HERSHEY_SIMPLEX, 2, (0,0,0), 3)
    cv2.putText(final_frame, 'Player : Computer', (150,250), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,0,0), 2)
    score_text = f'{player_score} : {computer_score}'
    cv2.putText(final_frame, score_text, (270,320), cv2.FONT_HERSHEY_SIMPLEX, 2, (0,0,255), 3)
    cv2.imshow('Rock Paper Scissors', final_frame)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

play_rps()